Copyright **`(c)`** 2023 Laura Amoroso and Arturo Adelfio

This is the implementation of some of the Single State Methods applied to the Set Covering problem

In [115]:
from itertools import product
from random import random, randint, choice, seed, choice
import numpy as np
from scipy import sparse
from functools import reduce
from collections import namedtuple
import math
from copy import copy

In [116]:
def make_set_covering_problem(num_points, num_sets, density):
    """Returns a sparse array where rows are sets and columns are the covered items"""
    seed(num_points*2654435761+num_sets+density)
    sets = sparse.lil_array((num_sets, num_points), dtype=bool)
    for s, p in product(range(num_sets), range(num_points)):
        if random() < density:
            sets[s, p] = True
    for p in range(num_points):
        sets[randint(0, num_sets-1), p] = True
    return sets

# Halloween Challenge

Find the best solution with the fewest calls to the fitness functions for:

* `num_points = [100, 1_000, 5_000]`
* `num_sets = num_points`
* `density = [.3, .7]` 

In [126]:
num_points=100
num_sets=100
x = make_set_covering_problem(num_points, num_sets, .3)
counter=0
print("Element at row=42 and column=42:", x[42,42])



Element at row=42 and column=42: True


In [118]:

def fitness(state):
    global counter
    counter+=1
    cost = sum(state)
    l=np.empty((num_sets,num_points),dtype=bool)
    not_taken=np.empty((num_sets,num_points),dtype=bool)


    for j in range(num_sets):
        for i in range(num_points):
            if(state[j]):
                l[j,i]=x[j,i]
            else:
                not_taken[j,i]=x[j,i]

    
    already_covered = reduce(
        np.logical_or,
        [l[i] for i in range(num_sets) if state[i]],
        np.array([False for _ in range(num_sets)]),
    )
   
    valid = np.sum(
        already_covered
    )
    
    
    #new_metric=max(sum(np.logical_not(np.logical_xor(not_taken[i],already_covered ))) for i in range(num_sets) if not state[i])
    #print(valid,-cost)
    return valid, -cost if valid else 0

used_indeces=[]
def tweak(state):
    global used_indeces
    new_state = copy(state)

    index = randint(0, num_sets - 1)

    new_state[index] = not new_state[index]
    
    return new_state

We implemented a new tweak function that add the not yet taken set with the highest number of true elements and removes the one with the highest number of false ones

In [119]:
def new_tweak(state):
    new_state = copy(state)

    taken=[]
    not_taken=[]
    covered=np.empty((num_sets,num_points),dtype=bool)
    uncovered=np.empty((num_sets,num_points),dtype=bool)

    for j in range(num_sets):
        for i in range(num_points):
            if(state[j]):
                taken.append(j)
                covered[j,i]=x[j,i]
            else:
                not_taken.append(j)
                uncovered[j,i]=x[j,i]
    

    index=choice(not_taken)

    new_state[index] = True
    for i in range(num_points):
        covered[index,i]=x[index,i]

    sum=0
    current_max=0
    current_max_index=0
    #I removed the set with the highest number of false elements
    for j in range(num_sets):
        for i in range(num_points):
            if covered[j,i]==False:
                sum+=1
        if sum>current_max:
            current_max=sum
            current_max_index=j
    new_state[current_max_index] = False

    return new_state


HILL CLIMBING

In [120]:
def hill_climbing():
    global counter
    counter=0
    current_state = [choice([False, False, False, False, False, False]) for _ in range(num_sets)]
    ended=False
    is_better=True
    not_improving=0
    while (not ended) or not_improving>100:
        new_state = new_tweak(current_state)

        new_f=fitness(new_state)
        
        new_covered_points=new_f[0]

        if new_covered_points==num_points:
            ended=True

        is_better=new_f>fitness(current_state)

        if is_better:
            not_improving=not_improving-1
            current_state = new_state
        else:
            not_improving+=1
    print( f"Solved in {counter:,} steps")
    print("final solution", fitness(current_state))

hill_climbing()

Solved in 38 steps
final solution (1000, -16)


ITERATED LOCAL SEARCH

In [121]:
def hill_climbing_ils(current_state): 
    
    ended=False
    is_better=True
    not_improving=0

    while (not ended) or not_improving>100:
       
        new_state = new_tweak(current_state)

        new_f=fitness(new_state)
        
        new_covered_points=new_f[0]

        if new_covered_points==num_points:
            ended=True

        is_better=new_f>fitness(current_state)

        if is_better:
            not_improving=not_improving-1
            current_state = new_state
        else:
            not_improving=not_improving+1
    
    return current_state


In [122]:
def iterated_local_search():
    global counter
    counter=0
    current_state = [choice([False, False, False, False, False, False]) for _ in range(num_sets)]
    best_solution = current_state
   
    
    for i in range(5):
        print(i)
        current_solution = hill_climbing_ils(tweak(best_solution))
    
        if  fitness(current_solution) > fitness(best_solution):
            best_solution = current_solution
           
        print(fitness(best_solution))
    print( f"Solved in {counter:,} steps")
    print("final solution", fitness(best_solution))
    
iterated_local_search()

0
(1000, -20)
1
(1000, -20)
2
(1000, -20)
3
(1000, -20)
4
(1000, -20)
Solved in 75 steps
final solution (1000, -20)


We tried a version of the iterated local search that pertubates the best solution, but it did not improve our solutions

In [123]:
def iterated_local_search_perturbated():
    global counter
    counter=0
    current_state = [choice([False, False, False, False, False, False]) for _ in range(num_sets)]
    best_solution = current_state
    perturbation = current_state
    
    for i in range(5):
        print(i)
        current_solution = hill_climbing_ils(perturbation)
    
        if  fitness(current_solution) > fitness(best_solution):
            best_solution = current_solution
            random = np.random.random((num_sets,))<.1
            concatenated= [best_solution,random]
            perturbation =reduce(
                            np.logical_or, 
                           concatenated, np.array([False for _ in range(num_sets)]))
            print('random', random)
            print('best_solution', best_solution)
            print('perturbation', perturbation)
            print('random', random)
            print('len', len(random))
        print(fitness(best_solution))
    print(fitness(best_solution))

#iterated_local_search_perturbated()
    

TABU SEARCH

In [129]:
def is_valid(sol, current_state):
    return np.sum(sol)>0

#solved with 100
def tabu_search():
    global counter
    counter=0
    tabu_list=[]
    current_state = [choice([False, False, False, False, False, False]) for _ in range(num_sets)]
    
    for step in range(100):   
    
        tmp=(tweak(current_state) for _ in range(num_points))
        candidates=[(sol,fitness(sol)) for sol in tmp if is_valid(sol,current_state) and sol not in tabu_list]
        
        if not candidates:
            continue;
        else:
            max_sol= max(candidates, key=lambda x: x[1])

            if(fitness(max_sol[0])>fitness(current_state)):
                current_state=max_sol[0]
            tabu_list.append(current_state)
   
    current_state=max(tabu_list, key=lambda x:fitness(x))
    print( f"Solved in {counter:,} evaluations")
    print(fitness(current_state))
tabu_search()

Solved in 10,197 evaluations
(100, -6)


SIMULATED ANNEALING

In [127]:
#solved with 100
def simulated_annealing():
    global counter
    counter=0
    current_state = [choice([False, False, False, False, False, False]) for _ in range(num_sets)]
   
    t=num_points
    ended=False
    is_better=True
    iteration=0
    not_improving=0

    while (not ended) or not_improving < 1:
        iteration=iteration+1
        new_state = new_tweak(current_state)

        new_f=fitness(new_state)
        
        new_covered_points=new_f[0]

        is_better=new_f>fitness(current_state)

        if new_covered_points==100:
            ended=True
       
        if not is_better:
            not_improving=not_improving+1
            sottrazione = tuple(y-x  for x, y in zip(fitness(current_state), fitness(new_state)))

            minimum=-(abs(sottrazione[0])+abs(sottrazione[1]))
            if t<=0:
                p=0
            else:    
                esponente=minimum/t            
                p=math.exp(esponente)
                
           
        if is_better or random() < p:
            if is_better:
                not_improving=not_improving-1
            current_state = new_state
       
        alpha=num_points-new_covered_points
        t=t*0.5

    print( f"Solved in {counter:,} steps")
    print("finale state", fitness(current_state))

simulated_annealing()

Solved in 70 steps
finale state (100, -11)
